In [2]:
import os
os.getcwd()


'c:\\Krish\\Full GenAI course\\openai_basic\\4.MedicalChatbot\\naturopathychatbot'

In [3]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone, faiss
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import CTransformers
from langchain_community.vectorstores import FAISS
from langchain_pinecone import PineconeVectorStore


In [4]:
api_key = os.environ.get('PINECONE_API_KEY') 
print(api_key)

1d3da372-b023-4cfd-baea-ed43452239d0


In [5]:
def load_pdf(data):
    loader=DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    docs=loader.load()
    return docs

In [6]:
extracted_data=extracted_data = load_pdf("data/")

In [7]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [8]:
text_chunks=text_split(extracted_data)
print(f'length of my chunk: {len(text_chunks)}')

length of my chunk: 2266


In [9]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings
embeddings = download_hugging_face_embeddings()

In [10]:
query_result = embeddings.embed_query("Hello world")
print(len(query_result))
query_result

384


[-0.034477200359106064,
 0.031023219227790833,
 0.006734990049153566,
 0.026108969002962112,
 -0.03936203196644783,
 -0.16030253469944,
 0.06692396104335785,
 -0.006441513076424599,
 -0.04745054617524147,
 0.014758806675672531,
 0.07087530195713043,
 0.055527545511722565,
 0.01919337920844555,
 -0.02625131979584694,
 -0.010109491646289825,
 -0.026940487325191498,
 0.02230740524828434,
 -0.02222667634487152,
 -0.1496926248073578,
 -0.017493046820163727,
 0.0076762777753174305,
 0.054352302104234695,
 0.0032544457353651524,
 0.03172589838504791,
 -0.08462142944335938,
 -0.029405998066067696,
 0.05159560963511467,
 0.04812406003475189,
 -0.003314807778224349,
 -0.05827923119068146,
 0.04196930676698685,
 0.022210698574781418,
 0.128188818693161,
 -0.02233896031975746,
 -0.011656238697469234,
 0.06292834132909775,
 -0.03287627175450325,
 -0.09122605621814728,
 -0.03117539919912815,
 0.05269959568977356,
 0.0470348559319973,
 -0.08420301973819733,
 -0.030056145042181015,
 -0.020744780078530

In [13]:
import os
from pinecone import Pinecone

# initialize connection to pinecone (get API key at app.pinecone.io)


# configure client
pc = Pinecone(api_key=api_key)

from pinecone import ServerlessSpec

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

index_name = 'mchatbot'

index = pc.Index(index_name)
index.describe_index_stats()


{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 6798}},
 'total_vector_count': 6798}

In [11]:
index_name='mchatbot'

In [12]:
#index=pinecone.Index(api_key=pc,index_name=index_name, host='https://mchatbot-9sx78ii.svc.aped-4627-b74a.pinecone.io')

In [13]:
docsearch=PineconeVectorStore.from_documents(documents=text_chunks, embedding=embeddings, index_name=index_name)

In [14]:
query='what are the treatments for stomach ulcers?'
docs=docsearch.similarity_search(query, k=3)
print("Result", docs)



Result [Document(page_content='NATURE CURE\n\xa0 Peptic Ulcer', metadata={'page': 180.0, 'source': 'data\\acompletehandbookofnaturecure-211207173053.pdf'}), Document(page_content='NATURE CURE\n\xa0 Peptic Ulcer', metadata={'page': 180.0, 'source': 'data\\acompletehandbookofnaturecure-211207173053.pdf'}), Document(page_content='NATURE CURE\n\xa0 Peptic Ulcer', metadata={'page': 180.0, 'source': 'data\\acompletehandbookofnaturecure-211207173053.pdf'})]


In [15]:
for i in range(len(docs)):
    print(docs[i].page_content)

NATURE CURE
  Peptic Ulcer
NATURE CURE
  Peptic Ulcer
NATURE CURE
  Peptic Ulcer


In [16]:
response_json={"symptoms": "symptoms", "treatment":"treatment"}

In [24]:

prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [25]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context","question"])
chain_type_kwargs={"prompt": PROMPT}

In [27]:
llm=CTransformers(model="llama-2-7b-chat.ggmlv3.q8_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [28]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 3}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [30]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])



Response :  To treat Irritable Bowel Syndrome (IBS), it is important to cleanse the system of toxic waste through proper dietary and other natural methods. The patient should fast for three to five days, consuming only water and orange juice during this time. Daily bowel cleansing is also recommended using a fiber-rich diet and drinking plenty of water. Additionally, stress management of the use stress management of the patient may consider incorporating certain herbalancingto reduce oraloeother natural remeduciding relaxation theraproots may other natural remeducs stress management of course of great results from time- managing and relaxation can help your healthyoga and relaxation therapathlet'stressential oat- reducing stress reduction of the patient should consider incorporating certain herbalancing to reduce oralmondays such therapither probiotaspironionizing stress management of great care should try to further reducing stress management of the use of the patient may other natura

KeyboardInterrupt: 

In [149]:
response_json={"symptoms": "symptoms", "treatment":"treatment"}

prompt_template='''
You are an expert Naturopathy doctor. Given the {disease}, it is your job to list down the symptoms and suggest treatment. 
Make sure to format your responses in the format: {response_json}     

# Context: {context}
# Disease: {disease}
'''

PROMPT=PromptTemplate(template=prompt_template, input_variables=["context","disease", "response_json"])
chain_type_kwargs={"prompt": PROMPT}

qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 3}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)


import json
disease = "Irritable Bowel Syndrome"
response_json = json.dumps({"symptoms": "symptoms", "treatment": "treatment"})

result = qa({
    "disease": disease,
    "query": "What are the symptoms and treatment?",
    "response_json": response_json
})

ValueError: Missing some input keys: {'disease', 'response_json'}

In [21]:
response_json={"symptoms": "symptoms here", "treatment":"treatment"}

template='''
You are an expert Naturopathy doctor. Given the {disease}, it is your job to list down the symptoms and suggest treatment. 
Make sure to format your responses in the format: {response_json}     
'''
prompt_template=PromptTemplate(input_variables=['disease', 'response_json'], template=template)

from langchain.chains import LLMChain

treatment_chain=LLMChain(llm=llm, prompt=prompt_template)


In [23]:
import json
disease = "Irritable Bowel Syndrome"
info=treatment_chain.invoke(input={'disease':disease, 'response_json':json.dumps(response_json)})

In [163]:
info['text']

'Symptoms:\n1. Abdominal pain or cramping\n2. Diarrhea or constipation\n3. Bloating or gas\n4. Feeling of urgency to have a bowel movement\n5. Nausea and vomiting\n6. Fatigue\n7. Loss of appetite\n8. Dehydration\n9. Blood in stool or black tarry stools\n10. Weight loss\n\nTrecommendations:\n\n1. Dietary changes: Avoid gluten, dairy, and processed foods, which can irritate the gut and worsen symptoms. Instead, focus on eating a diet rich in whole, unprocessed foods such as fruits, vegetables, lean proteins, and whole grains.\n2. Probiotics: Take probiotics supplements or eat probiotic-rich foods like yogurt, kefir, sauerkraut, and kimchi to support the growth of beneficial gut bacteria.\n3. Stress management: Practice stress-reducing techniques such as meditation, deep breathing, or yoga to help manage stress, which can exacerbate IBS symptoms.\n4. Sleep: Aim for 7-8 hours of sleep per night to help regulate digestion and reduce symptoms.\n5. Herbal remedies: Consider taking herbal reme

In [56]:
query="How do we treat Acne?"
result=qa({"query": query})
result['result']

'Lemon has been found to be effective in removing pimples and acne by applying it regularly. Grated cucumber, oatmeal cooked in milk, and cooked, creamed carrots used externally have also been found to be helpful in treating acne.'

In [74]:
query="How do we treat IBS?"
result=qa({"query": query})
result['result']

'Treatment of IBS involves several natural methods to cleanse the system of toxic waste, including fasting for 3-5 days, drinking only water and orange juice during this time, and cleansing the bowels daily with warm water.'